## Read main xlsx file.

In [1]:
import pandas as pd

df = pd.read_excel('Umrahbooking.xlsx')

print(df.head())

                                Ratehawk Code  \
0  makkah_clock_royal_tower__a_fairmont_hotel   
1                            swissotel_makkah   
2                      rosh_rayhaan_by_rotana   
3           anwar_al_madinah_moevenpick_hotel   
4                      pullman_zamzam_madina_   

                          Hotel name                             Address  \
0  Fairmont Makkah Clock Royal Tower   King Abdul Aziz Endownment, Mecca   
1                   Swissôtel Makkah    King Abdul Aziz Endowment, Mecca   
2             Rosh Rayhaan by Rotana             Al Olaya Street, Riyadh   
3   Anwar Al Madinah Mövenpick Hotel  Central Zone, Al Khalidiya, Medina   
4              Pullman Zamzam Madina      Amr Bin Al Gmoh Street, Medina   

  Country Iso Hotel country Hotel city  Star rating   latitude  longitude  \
0          SA  Saudi Arabia      Mecca            5  21.418213  39.825176   
1          SA  Saudi Arabia      Mecca            5  21.418900  39.827100   
2          SA  Sa

### Only give headers

In [13]:
import pandas as pd

df = pd.read_excel('Umrahbooking.xlsx', nrows=0)

print(df.columns.tolist())

['Ratehawk Code', 'Hotel name', 'Address', 'Country Iso', 'Hotel country', 'Hotel city', 'Star rating', 'latitude', 'longitude', 'Mapped', 'giata_present']


### Add columns in xlsx file

In [10]:
# Get existing column names and add 'giata_present'
columns = df.columns.tolist() + ['giata_present']

print(columns)

['Ratehawk Code', 'Hotel name', 'Address', 'Country Iso', 'Hotel country', 'Hotel city', 'Star rating', 'latitude', 'longitude', 'Mapped', 'giata_present', 'giata_present']


In [11]:
import pandas as pd

df = pd.read_excel('Umrahbooking.xlsx', nrows=0)

print(df.columns.tolist())

['Ratehawk Code', 'Hotel name', 'Address', 'Country Iso', 'Hotel country', 'Hotel city', 'Star rating', 'latitude', 'longitude', 'Mapped', 'giata_present']


### Add columns permanetly

In [10]:
import pandas as pd

df = pd.read_excel('Umrahbooking.xlsx')

df['giata_present'] = None  

df.to_excel('Umrahbooking.xlsx', index=False)

print("Column 'giata_present' added successfully!")


Column 'ratehawk_present' added successfully!


### Get first row first column.

In [14]:
import pandas as pd

# Read the first row and first column only
df = pd.read_excel('Umrahbooking.xlsx', usecols=[0], nrows=1)

# Extract the value
first_value = df.iloc[0, 0]

print("First row, first column value:", first_value)


First row, first column value: makkah_clock_royal_tower__a_fairmont_hotel


### Check and change status for giata id present or not

In [ ]:
import pandas as pd
import requests
import xml.etree.ElementTree as ET
import time
from dotenv import load_dotenv
import os

load_dotenv()


GIATA_KEY = os.getenv("GIATA_API_key")

# Read the Excel file
df = pd.read_excel('Umrahbooking.xlsx')

# Ensure 'giata_present' column is of type string
df['giata_present'] = df['giata_present'].astype(str)

headers = {
    'Authorization': f'Basic {GIATA_KEY}'
}


# Iterate over each row in the DataFrame
for index, row in df.iterrows():
    ratehawk_code = row['Ratehawk Code']
    url = f"https://multicodes.giatamedia.com/webservice/rest/1.0/properties/gds/ratehawk2/{ratehawk_code}"
    
    try:
        # Send POST request
        response = requests.post(url, headers=headers)
        response.raise_for_status()  # Raise exception for HTTP errors
        
        # Parse XML response
        root = ET.fromstring(response.content)
        property_element = root.find('property')
        
        # Check for giataId attribute and set giata_id to that value if present
        if property_element is not None and 'giataId' in property_element.attrib:
            giata_id = property_element.attrib['giataId']
        else:
            giata_id = 'no'
            
    except requests.exceptions.HTTPError as http_err:
        print(f"HTTP error occurred for {ratehawk_code}: {http_err}")
        giata_id = 'error'
    except ET.ParseError as xml_err:
        print(f"XML parse error for {ratehawk_code}: {xml_err}")
        giata_id = 'error'
    except Exception as err:
        print(f"Other error occurred for {ratehawk_code}: {err}")
        giata_id = 'error'
    
    # Update the 'giata_present' column with the actual giata id or fallback value
    df.at[index, 'giata_present'] = giata_id
    
    # Save the updated DataFrame immediately after processing each row
    df.to_excel('Umrahbooking_updated.xlsx', index=False)
    
    # Print a success message for the processed row
    print(f"Processed {ratehawk_code} successfully with giata id: {giata_id}")
    
    # Delay to avoid hitting rate limits
    time.sleep(1)

print("Processing complete. Updated file saved as 'Umrahbooking_updated.xlsx'")


Processed makkah_clock_royal_tower__a_fairmont_hotel successfully with giata id: 238369
Processed swissotel_makkah successfully with giata id: 282258
Processed rosh_rayhaan_by_rotana successfully with giata id: 571095
Processed anwar_al_madinah_moevenpick_hotel successfully with giata id: 138287
Processed pullman_zamzam_madina_ successfully with giata id: 427649
Processed hotel_pullman_zamzam_makkah successfully with giata id: 129794
Processed hyatt_regency_makkah successfully with giata id: 431847
Processed address_jabal_omar_makkah successfully with giata id: 1375247
Processed voco_makkah successfully with giata id: 1193729
Processed millennium_makkah_al_naseem successfully with giata id: 1039856
Processed joudyan_hotel successfully with giata id: 1332109
Processed riyadh_marriott_hotel successfully with giata id: 431846
Processed aloft_dhahran successfully with giata id: 594100
Processed shaza_al_madina successfully with giata id: 244304
Processed suite_novotel_riyadh_olaya successf

### Here only take where Mapped = no

In [1]:
import pandas as pd
import requests
import xml.etree.ElementTree as ET
import time
from dotenv import load_dotenv
import os

load_dotenv()


GIATA_KEY = os.getenv("GIATA_API_key")

# Read the Excel file
df = pd.read_excel('Umrahbooking.xlsx')

# Ensure 'giata_present' column is of type string
df['giata_present'] = df['giata_present'].astype(str)

headers = {
    'Authorization': f'Basic {GIATA_KEY}'
}


for index, row in df.iterrows():
    if str(row['Mapped']).strip().lower() != 'no':
        print(f"Skipping {row['Ratehawk Code']} because Mapped is not 'no'")
        continue
    
    ratehawk_code = row['Ratehawk Code']
    url = f"https://multicodes.giatamedia.com/webservice/rest/1.0/properties/gds/ratehawk2/{ratehawk_code}"
    
    try:
        response = requests.post(url, headers=headers)
        response.raise_for_status()  # Raise exception for HTTP errors
        
        root = ET.fromstring(response.content)
        property_element = root.find('property')
        
        # Check for giataId attribute and set giata_id to that value if present
        if property_element is not None and 'giataId' in property_element.attrib:
            giata_id = property_element.attrib['giataId']
        else:
            giata_id = 'no'
            
    except requests.exceptions.HTTPError as http_err:
        print(f"HTTP error occurred for {ratehawk_code}: {http_err}")
        giata_id = 'error'
    except ET.ParseError as xml_err:
        print(f"XML parse error for {ratehawk_code}: {xml_err}")
        giata_id = 'error'
    except Exception as err:
        print(f"Other error occurred for {ratehawk_code}: {err}")
        giata_id = 'error'
    
    # Update the 'giata_present' column with the actual giata id or fallback value
    df.at[index, 'giata_present'] = giata_id
    
    # Save the updated DataFrame immediately after processing each row
    df.to_excel('Umrahbooking_updated.xlsx', index=False)
    
    # Print a success message for the processed row
    print(f"Processed {ratehawk_code} successfully with giata id: {giata_id}")
    
    # Delay to avoid hitting rate limits
    time.sleep(1)

print("Processing complete. Updated file saved as 'Umrahbooking_updated.xlsx'")


Skipping makkah_clock_royal_tower__a_fairmont_hotel because Mapped is not 'no'
Skipping swissotel_makkah because Mapped is not 'no'
Skipping rosh_rayhaan_by_rotana because Mapped is not 'no'
Skipping anwar_al_madinah_moevenpick_hotel because Mapped is not 'no'
Skipping pullman_zamzam_madina_ because Mapped is not 'no'
Skipping hotel_pullman_zamzam_makkah because Mapped is not 'no'
Skipping hyatt_regency_makkah because Mapped is not 'no'
Skipping address_jabal_omar_makkah because Mapped is not 'no'
Skipping voco_makkah because Mapped is not 'no'
Skipping millennium_makkah_al_naseem because Mapped is not 'no'
Skipping joudyan_hotel because Mapped is not 'no'
Skipping riyadh_marriott_hotel because Mapped is not 'no'
Skipping aloft_dhahran because Mapped is not 'no'
Skipping shaza_al_madina because Mapped is not 'no'
Skipping suite_novotel_riyadh_olaya because Mapped is not 'no'
Skipping odst_al_madinah_hotel because Mapped is not 'no'
Skipping al_kiswah_towers_hotel because Mapped is not 

### Count total ids find.

In [6]:
import pandas as pd

# Load the Excel file
df = pd.read_excel('Umrahbooking_updated.xlsx')

# Calculate counts
no_count = (df['giata_present'] == 'no').sum()
nan_count = df['giata_present'].isna().sum()
others_count = len(df) - no_count - nan_count

# Display results
print(f"Number of 'no' entries: {no_count}")
print(f"Number of NaN entries: {nan_count}")
print(f"Number of giata id find: {others_count}")

Number of 'no' entries: 1111
Number of NaN entries: 7476
Number of giata id find: 185


### Here show all row where giata id is present

In [4]:
import pandas as pd

# Load the Excel file
df = pd.read_excel('Umrahbooking_updated.xlsx')

# Calculate counts
no_count = (df['giata_present'] == 'no').sum()
nan_count = df['giata_present'].isna().sum()
others_count = len(df) - no_count - nan_count

print(f"Number of 'no' entries: {no_count}")
print(f"Number of NaN entries: {nan_count}")
print(f"Number of giata id find: {others_count}")

other_rows = df[(df['giata_present'].notna()) & (df['giata_present'] != 'no')]

# Display the filtered rows
print("\nRows with 'other' entries:")
print(other_rows)



Number of 'no' entries: 1111
Number of NaN entries: 7476
Number of other entries (e.g., 'yes', 'error'): 185

Rows with 'other' entries:
                                     Ratehawk Code  \
257                      narcissus_the_royal_hotel   
428                             saja_hotels_makkah   
467                                     ross_hotel   
571        le_park_concord_rabwah_boutique_hotel_2   
601                    nova_m_hotel_edge_by_rotana   
...                                            ...   
8720  bab_samhan_a_luxury_collection_hotel_diriyah   
8725              ram_jeddah_serviced_apartments_2   
8740                                  masat_albadr   
8747                                 green_view_12   
8760                           sama_al_amani_hotel   

                                          Hotel name  \
257                        Narcissus The Royal Hotel   
428                               Saja Hotels Makkah   
467                                       Ross

### Creat all id list where giata id only find.

In [7]:
import pandas as pd

def get_giata_ids(filename):
    """
    Loads the Excel file and returns a list of giata id values from the 'giata_present'
    column where the value is not 'no' and not NaN.
    """
    # Load Excel file
    df = pd.read_excel(filename)
    
    # Filter rows: only keep those where giata_present is not NaN and not 'no'
    filtered = df[(df['giata_present'].notna()) & (df['giata_present'] != 'no')]
    
    # Extract the list of giata id values
    giata_ids = filtered['giata_present'].tolist()
    return giata_ids

# Example usage:
if __name__ == "__main__":
    giata_ids = get_giata_ids('Umrahbooking_updated.xlsx')
    # Print each giata id on a new line
    for giata in giata_ids:
        print(giata)


1428454
1433106
1427254
1316096
1432054
1445377
1432067
1433340
1433141
1294104
1416815
1430632
137490
720456
646988
928171
543839
1436629
1428455
1434876
1438113
107275
1442809
962124
1442196
1364779
1430359
1433298
889460
1432634
1446436
1433043
1405993
1432565
1432574
1428330
1428238
275276
1432551
503909
883830
1428549
244242
1432557
1432069
1441938
1434218
1433419
718542
1441973
1432440
1434269
1332063
1364779
1294344
1433409
1441386
917914
1126273
1445380
624715
1112073
919661
1442798
1396796
1457720
898354
928978
1420212
1331921
987610
729974
1433008
1435374
1360253
889384
1441930
1308204
968991
1428456
1396763
957702
1434121
604930
707619
811090
1446500
424515
707619
1364828
1441958
1433043
629095
1305626
1432534
1442211
1434302
1433275
129781
1338485
1331848
1455320
1317202
877093
1025128
919114
972163
918874
1456541
1359996
1428584
1007317
1237367
752475
1445400
1442603
1451339
1432073
1433280
1432065
1364692
1364841
1445350
1428214
687950
957547
811267
707862
1416963
1434125

## Create new xlsx file follow 'NO' 
### Create a xlsx file where giata id is not present but client xlsx file give it not insert global mapping database.

In [17]:
import pandas as pd

# Load the Excel file
df = pd.read_excel('Umrahbooking_updated.xlsx')

# Filter rows where 'giata_present' is 'no'
no_rows = df[df['giata_present'] == 'no']

# Save to a new Excel file
no_rows.to_excel('not_find.xlsx', index=False)

print("File 'not_find.xlsx' has been created with rows where 'giata_present' is 'no'.")


File 'not_find.xlsx' has been created with rows where 'giata_present' is 'no'.


### Add a column for not finding giata tracking file.

In [18]:
import pandas as pd

df = pd.read_excel('not_find.xlsx')

df['ratehawk_present'] = None  

df.to_excel('not_find.xlsx', index=False)

print("Column 'ratehawk_present' added successfully!")


Column 'ratehawk_present' added successfully!


### Separet 'find_id' and 'not_find_id' in xlsx.

In [19]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy.sql import text

# Load the Excel file
df = pd.read_excel('not_find.xlsx')

# Add a new column for ratehawk presence
df['ratehawk_present'] = None

# Database connection setup (replace with your actual database URL)
DATABASE_URL = "mysql+pymysql://root:@localhost/csvdata01_02102024"
engine = create_engine(DATABASE_URL)
Session = sessionmaker(bind=engine)
session = Session()

def check_ratehawk_presence(ratehawk_id, session):
    query = text("SELECT * FROM ratehawk WHERE id = :ratehawk_id")
    result = session.execute(query, {'ratehawk_id': ratehawk_id}).fetchone()
    return 'find' if result is not None else 'not_find'

for index, row in df.iterrows():
    if str(row['Mapped']).strip().lower() != 'no':
        continue
    
    ratehawk_code = row['Ratehawk Code']
    
    presence = check_ratehawk_presence(ratehawk_code, session)
    df.at[index, 'ratehawk_present'] = presence
    print(f"Processed {ratehawk_code} with result: {presence}")

df.to_excel('not_find.xlsx', index=False)
print("Updated file saved as 'not_find.xlsx'")


Processed damac_towers_olaya_by_selsal with result: find
Processed holiday_suites_al_azizia with result: find
Processed rose_sharurah_serviced_apartments with result: find
Processed mirada_purple_al_waha with result: find
Processed arjan_park_hotel_al_aziziyah with result: find
Processed al_mabat_serviced_apartments with result: find
Processed wateel with result: not_find
Processed samaya_spring_hotel with result: not_find
Processed towlan_qurtuba_2 with result: find
Processed wissam_alhawra_hotel with result: find
Processed rose_plaza_3 with result: find
Processed karam_aldiafa_hotel_suites with result: find
Processed jawharat_qasr_al_safwa with result: find
Processed wissam_garden with result: find
Processed gp_resort with result: find
Processed kease_malqa_e1_elegance_ag29 with result: find
Processed mabaat_alyasmin_555 with result: find
Processed almiqat_ajyad_hotel with result: find
Processed arjaan_park_al_shifa_hotel with result: not_find
Processed yaqoot_al_madina_suites with r

### Find total ids and not find total ids information

In [20]:
import pandas as pd

df = pd.read_excel('not_find.xlsx')

find_count = (df['ratehawk_present'] == 'find').sum()
not_find_count = (df['ratehawk_present'] == 'not_find').sum()

print(f"Number of find_count: {find_count}")
print(f"Number of not_find_count: {not_find_count}")

Number of find_count: 176
Number of not_find_count: 935
